In [74]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [30]:
df_ratings = pd.read_csv('C:/Users/gordee-ext/OneDrive - Louis Dreyfus Company/Desktop/Netology/ml-latest-small/ratings.csv')
df_movies = pd.read_csv('C:/Users/gordee-ext/OneDrive - Louis Dreyfus Company/Desktop/Netology/ml-latest-small/movies.csv')
df_tags = pd.read_csv('C:/Users/gordee-ext/OneDrive - Louis Dreyfus Company/Desktop/Netology/ml-latest-small/tags.csv')

In [31]:
data = pd.merge(df_ratings, df_movies, on='movieId')

In [32]:
del data['timestamp']

In [33]:
df=pd.merge(data, df_tags, on='movieId')

In [34]:
del df['timestamp']
del df['userId_y']

In [38]:
df.head()

,userId_x,movieId,rating,title,genres,tag
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar
1,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar
2,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,fun
3,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar
4,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar


In [40]:
df['userId_x'].describe()

count    233213.000000
mean        309.688191
std         178.206387
min           1.000000
25%         156.000000
50%         309.000000
75%         460.000000
max         610.000000
Name: userId_x, dtype: float64

In [41]:
MAX_USER_ID = 610
MIN_USER_ID = 1

In [42]:
movie_tags = df['tag'].unique()

In [43]:
movie_tags = movie_tags.tolist()

In [47]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [49]:
movie_genres = [change_string(g) for g in df.genres.values]

In [63]:
movie_genres_st=[g.split(' ') for g in movie_genres]

In [70]:
movie_genres_st

[['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adven

In [71]:
myset = set(movie_genres_st)

TypeError: unhashable type: 'list'

In [69]:
myset

{'Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'}

In [36]:
movie_to_vector = {}

In [37]:
for movie in tqdm_notebook(movie_tags):
    movie_to_vector[movie] = np.zeros((MAX_USER_ID,))
    for r in df[df['tag'] == movie].iterrows():
        movie_to_vector[movie][r[1]['userId'] - MIN_USER_ID] = r[1]['rating']

In [43]:
def find_similar(movie, dist_func, top=10):
    distances = {}
    target_movie = movie_to_vector[movie]
    for m in movie_tags:
        distances[m] = dist_func(target_movie, movie_to_vector[m])
        
    distances_with_idx = [(i, distances[m]) for i, m in enumerate(movie_tags)]
    distances_with_idx = sorted(distances_with_idx, key=lambda t: t[1], reverse=False)
    distances_with_idx = distances_with_idx[:top]
    
    return [(movie_tags[i], d) for i, d in distances_with_idx]

In [44]:
from scipy.spatial.distance import cosine, euclidean, cityblock

In [45]:
find_similar('pixar', cityblock)

[('pixar', 0.0),
 ('darth vader', 5.0),
 ('luke skywalker', 5.0),
 ('Dinosaur', 5.0),
 ('Politics', 5.0),
 ('England', 5.0),
 ('I am your father', 5.0),
 ('ark of the covenant', 5.0),
 ('brainwashing', 5.0),
 ('Tolkein', 5.0)]

In [54]:
conda install -c conda-forge scikit-surprise

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\gordee-ext\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - scikit-surprise


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.3.9           |           py37_0         149 KB  conda-forge
    conda-4.7.11               |           py37_0         3.0 MB  conda-forge
    conda-package-handling-1.4.1|           py37_0         281 KB  conda-forge
    joblib-0.13.2              |             py_0         180 KB  conda-forge
    scikit-surprise-1.0.6      |py37h452e1ab_1001         590 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following NEW packages will be INSTALLED:

  conda-package-han~ conda-forge/win-64::conda-package-handling-1.4.1-py37_0
  joblib             



==> WARNING: A newer version of conda exists. <==
  current version: 4.6.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda


'ET' is not recognized as an internal or external command,
operable program or batch file.



Note: you may need to restart the kernel to use updated packages.


In [1]:
from surprise import Dataset, Reader
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise import accuracy

In [105]:
df_ratings.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [106]:
df_for_surpise = df[['tag','genres', 'rating']]

In [107]:
df_for_surpise.columns = ['tag','genres', 'rating']

In [108]:
df_for_surpise.head()

,tag,genres,rating
0,pixar,Adventure|Animation|Children|Comedy|Fantasy,4.0
1,pixar,Adventure|Animation|Children|Comedy|Fantasy,4.0
2,fun,Adventure|Animation|Children|Comedy|Fantasy,4.0
3,pixar,Adventure|Animation|Children|Comedy|Fantasy,4.0
4,pixar,Adventure|Animation|Children|Comedy|Fantasy,4.0


In [109]:
reader = Reader(rating_scale=(0.5, 5))
dataset = Dataset.load_from_df(df_for_surpise, reader)
trainset, testset = train_test_split(dataset, test_size=0.2)

In [115]:
from surprise.model_selection import cross_validate

In [110]:
algo = SVD()

In [111]:
algo.fit(trainset)

In [112]:
predictions = algo.test(testset)

In [113]:
accuracy.mae(predictions)

MAE:  0.7057


0.7056632699549058

In [79]:
from surprise.model_selection import KFold

In [80]:
kfold = KFold(5)

In [118]:
scores = []
for trainset, testset in tqdm_notebook(kfold.split(dataset)):
    algo = SVD()
    algo.fit(trainset)
    predictions = algo.test(testset)
    scores.append(accuracy.mae(predictions))

MAE:  0.7124
MAE:  0.7091
MAE:  0.7032
MAE:  0.7095
MAE:  0.7107



In [119]:
scores

[0.7123834874692027,
 0.709065945576804,
 0.703224820310323,
 0.7095237962367579,
 0.7107138426373437]